<a href="https://colab.research.google.com/github/Ibtisam-a/Project-Implementation/blob/master/Preconditioned_MCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from scipy.sparse import diags

#Our parameters
#number of internal nodes , unknowns, in each direction

N = 190  # our values: 190, 380, 760, 1520, 3040, 6080 and 12160

#number of internal nodes, unknowns, in each direction
n = N-2 
#step size 
h = 1/(n+1) 

#Matrix with zeros
U = np.zeros([n,n])

# x and y arrays between 0 and 1, spaced internal points
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Internal mesh without boundaries
X, Y = np.meshgrid(x, y, indexing='ij')

#our function 
F = np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)  

#Tridiagonal matrix T
diagonals = [[1.5],[-0.5],[-0.5]]
T = np.multiply(1, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute the exact solution on a specific time (0.00000000001) to be comapred 
 # 0.0000001, 0.000000025, 0.00000000625, 0.0000000015625, 0.00000000039062, 0.00000000009766, 0.000000000024415
Exact_sol = np.exp(-8*(np.pi*np.pi)*(0.0000001))*np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)




In [5]:
from scipy.sparse import kron
from scipy.sparse.linalg import cg
import time
import numpy as np
#Define parameters

def precon_mcg(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
    
    start_time = time.time()
    norm = np.sqrt(np.trace(np.matmul(T.transpose(),T)))
    alpha = -np.sqrt(n)/norm
    Y = np.linalg.inv((alpha*T - np.eye(n)))
    T_hat = np.eye(n) + 2*Y
    F_hat = 2*alpha*(np.matmul(np.matmul(Y,-F),Y))
    R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
    Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
    Z = Q    

    num_iters = 0
#     if (np.any(np.absolute(Z) > tol).any()) == True:
    while ((np.absolute(R) > tol).any() == True):
        gamma = np.trace(np.matmul(R.transpose(),R))/np.trace(np.matmul(Z.transpose(),Z))
        eta_0 = np.trace(np.matmul(R.transpose(),R))
        U = U + gamma*Z
        R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
        Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
        eta = np.trace(np.matmul(R.transpose(),R))/eta_0
        Z = Q + eta*Z
        num_iters += 1
    all_timess = time.time() - start_time
    return U, all_timess, num_iters

In [ ]:
import numpy as np
def compute_error(U,Exact_sol):
    n = len(U)
    # The maximum difference between both solutions
    err_max_difference = 0  
    # The average difference between both solutions 
    err_ave_difference = 0

#For loop to compute the error
    for i in range(0,n):
        for j in range(0,n):
            err_ave_difference += np.absolute(Exact_sol[i][j] - U[i][j])**2
            if np.absolute(Exact_sol[i][j] - U[i][j]) > err_max_difference:
                err_max_difference = np.absolute(Exact_sol[i][j] - U[i][j])       
    err_ave_difference = (err_ave_difference * h**2)**0.5
    return err_max_difference, err_ave_difference

#Solving our system
U, all_timess, num_iters = precon_mcg(T, F,1e-9)
err_max_difference, err_ave_difference = compute_error(U, Exact_sol)

print('Time taken:', all_timess)
print('Maximum diffence:', err_max_difference, '\n', 'Average difference:', err_ave_difference)
